In [5]:
import torch
from transformers import AutoModelForImageClassification, AutoFeatureExtractor
from torch.quantization import quantize_dynamic, get_default_qconfig, prepare, convert, quantize_fx
import torch.nn.utils.prune as prune

In [7]:
# 1. Load the pre-trained model and tokenizer
model_name = './mobilenet_v2_affectnethq-fer2013_model'
output_dir = "./mobilenet_v2_affectnethq-fer2013_quantized_pruned" #Replace with your output directory
model = AutoModelForImageClassification.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)


/home/lundm/.local/lib/python3.10/site-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:28: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(


In [ ]:
# 2. Pruning
pruning_amount = 0.1  # Adjust as needed
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=pruning_amount)

        # Manually remove pruning buffers
        for buffer_name in list(module.named_buffers()):
            if "weight_mask" in buffer_name or "weight_orig" in buffer_name:
                delattr(module, buffer_name[0])

In [ ]:
# 3. Quantization (Dynamic Example)
#quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

# 3. Quantization (Static Example - Requires Calibration)
model.eval()
model.qconfig = get_default_qconfig('fbgemm')
torch.quantization.prepare(model, inplace=True)

def calibrate(model, feature_extractor, num_calibration_steps=10):
    for _ in range(num_calibration_steps):
        random_input = torch.randn(1, 3, 224, 224)
        normalized_input = (random_input - random_input.min()) / (random_input.max() - random_input.min())
        inputs = feature_extractor(images=normalized_input, return_tensors="pt")
        model(**inputs)

calibrate(model, feature_extractor)
quantized_model = torch.quantization.convert(model)


# 3. Quantization (FX Example)
#model.eval()
#qconfig = get_default_qconfig("fbgemm")
#qconfig_dict = {"": qconfig}
#prepared_model = prepare(model, qconfig_dict)

#def calibrate_fx(prepared_model, tokenizer, num_calibration_steps=10):
#    for _ in range(num_calibration_steps):
#        inputs = tokenizer("This is a sample sentence.", return_tensors="pt")
#        prepared_model(**inputs)

#calibrate_fx(prepared_model, tokenizer)
#quantized_model = convert(prepared_model)



In [16]:
# 4. Save the quantized and pruned model
quantized_model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)

print(f"Quantized and pruned model saved to {output_dir}")

AttributeError: 'NoneType' object has no attribute 'data_ptr'